In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [ ]:
data = {
    'age': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'km': [10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000],
    'price': [2000000, 1800000, 1600000, 1400000, 1200000, 1000000, 900000, 800000, 700000, 6000000]
}

# Создание DataFrame
data = pd.DataFrame(data)

In [ ]:
# Загрузка данных
X = data[['age', 'km']].values
y = data['price'].values

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Преобразование в тензоры
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

In [ ]:
X_train_tensor.shape[1]

2

In [ ]:
y_train_tensor.shape[1]

1

Предложите вариант обучения модели

In [ ]:
## альтернативно - через nn.Module:
# class CarPrice(nn.Module):
#     def __init__(self):
#         super(CarPrice, self).__init__()
#         self.fc1 = nn.Linear(2, 64)  # 2 входных признака
#         self.fc2 = nn.Linear(64, 32)
#         self.fc3 = nn.Linear(32, 1)   # 1 выходной признак (цена)

#     def forward(self, x):
#         x = torch.relu(self.fc1(x)) # меняем функцию активации
#         x = torch.relu(self.fc2(x)) # меняем функцию активации
#         x = self.fc3(x)
#         return x
# model = CarPrice()

In [ ]:
model = nn.Sequential(
    nn.Linear(2, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
)

In [ ]:
# Инициализация модели, функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [ ]:
# Обучение
num_epochs = 3000
for epoch in range(num_epochs):
    model.train()

    # Прямой проход
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor) # вычисление значения функции потерь

    # Обратный проход и оптимизация
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/3000], Loss: 1113278382080.0000
Epoch [200/3000], Loss: 833302364160.0000
Epoch [300/3000], Loss: 624117415936.0000
Epoch [400/3000], Loss: 377519570944.0000
Epoch [500/3000], Loss: 211459686400.0000
Epoch [600/3000], Loss: 124224847872.0000
Epoch [700/3000], Loss: 64320532480.0000
Epoch [800/3000], Loss: 22814269440.0000
Epoch [900/3000], Loss: 2553776896.0000
Epoch [1000/3000], Loss: 959020800.0000
Epoch [1100/3000], Loss: 818467200.0000
Epoch [1200/3000], Loss: 697153792.0000
Epoch [1300/3000], Loss: 593015808.0000
Epoch [1400/3000], Loss: 503419840.0000
Epoch [1500/3000], Loss: 426232864.0000
Epoch [1600/3000], Loss: 360521120.0000
Epoch [1700/3000], Loss: 303942848.0000
Epoch [1800/3000], Loss: 255828064.0000
Epoch [1900/3000], Loss: 215089024.0000
Epoch [2000/3000], Loss: 180406464.0000
Epoch [2100/3000], Loss: 150942768.0000
Epoch [2200/3000], Loss: 126144192.0000
Epoch [2300/3000], Loss: 105231440.0000
Epoch [2400/3000], Loss: 87529504.0000
Epoch [2500/3000], Loss: 7

In [ ]:
model.eval()
with torch.no_grad():
    predicted = model(X_test_tensor)
    test_loss = criterion(predicted, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.2f}')

Test Loss: 3460540399616.00


In [ ]:
# Пример новых данных о автомобиле
new_data = [[3, 50000]]  # Например, 3 года, 50000 км пробега

# Стандартизация новых данных
new_data_scaled = scaler.transform(new_data)

# Преобразование в тензор
new_data_tensor = torch.FloatTensor(new_data_scaled)

# Отключение градиентов для предсказания
with torch.no_grad():
    predicted_price = model(new_data_tensor)

# Вывод предсказанной цены
print(f'Предсказанная стоимость автомобиля: {predicted_price.item():.2f}')

Предсказанная стоимость автомобиля: 1398894.88


In [ ]:
# Сохранение модели
torch.save(model, 'car_price_model.pth')

# сохранение весов модели
# torch.save(model.state_dict(), 'car_price_model.pth')

In [ ]:
# Загрузка состояния модели
model_loaded = torch.load('car_price_model.pth')

# Переключение модели в режим оценки
model_loaded.eval()

<ipython-input-15-6ded6e38a689>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_loaded = torch.load('car_price_model.pth')


Sequential(
  (0): Linear(in_features=2, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=1, bias=True)
)

In [ ]:
# Пример новых данных о автомобиле
new_data = [[3, 50000]]  # Например, 3 года, 50000 км пробега

# Стандартизация новых данных
new_data_scaled = scaler.transform(new_data)

# Преобразование в тензор
new_data_tensor = torch.FloatTensor(new_data_scaled)

# Отключение градиентов для предсказания
with torch.no_grad():
    predicted_price = model_loaded(new_data_tensor)

# Вывод предсказанной цены
print(f'Предсказанная стоимость автомобиля: {predicted_price.item():.2f}')

Предсказанная стоимость автомобиля: 1398894.88


Есть прототип приложения на streamlit
Задача применить модель pyTorch


In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import torch
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = {
    'age': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'km': [10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000],
    'price': [2000000, 1800000, 1600000, 1400000, 1200000, 1000000, 900000, 800000, 700000, 6000000]
}

# Создание DataFrame
data = pd.DataFrame(data)

# Загрузка данных
X = data[['age', 'km']].values
y = data['price'].values

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


model_loaded = torch.load('car_price_model.pth')

# Заголовок приложения
st.title("Оценка стоимости автомобиля")

# Ввод данных пользователем
age = st.number_input("Возраст автомобиля (в годах)", min_value=0, max_value=20, value=1)
km = st.number_input("Пробег автомобиля (в километрах)", min_value=0, value=10000,step=1000)

# Кнопка для расчета стоимости
if st.button("Определить стоимость"):
    input_data = np.array([[age, km]])

    # Стандартизация новых данных
    new_data_scaled = scaler.transform(input_data)

    # Преобразование в тензор
    new_data_tensor = torch.FloatTensor(new_data_scaled)

    # Отключение градиентов для предсказания
    with torch.no_grad():
        predicted_price = model_loaded(new_data_tensor)

    st.write(f"Оценочная стоимость автомобиля: {predicted_price[0].numpy()[0]:,.2f} руб.")

Writing app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.125.16.45


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.16.45:8501

⠴⠦⠧⠇⠏Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧your url is: https://deep-baboons-film.loca.lt
/content/app.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializat